<a href="https://colab.research.google.com/github/Palaeoprot/PRIDE/blob/main/PRIDE_metadata_aggregator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PRIDE Metadata Aggregator

## Overview
This script aggregates metadata from multiple PRIDE (PRoteomics IDEntifications) database project files into a single consolidated CSV file. It's designed to work within Google Colab and integrates with the broader ZooMS data analysis pipeline.

## Features
- Automatically finds and processes all metadata JSON files in the specified directory
- Flattens nested JSON structures into a tabular format
- Handles various data types (lists, nested dictionaries, dates)
- Creates timestamped CSV exports for version control
- Provides detailed column information and data summaries
- Integrates with Google Drive for storage

## Prerequisites
- Google Colab environment
- Access to Google Drive with ZooMS_Data/PRIDE directory
- Python packages: pandas, tqdm

## Configuration Parameters
```python
shared_drive_base_dir_str = "/content/drive/Shareddrives/ZooMS_Data/PRIDE"
spreadsheet_id = '127K6zdl5y46DRqUwRr-V32nUDoceaddbhG9XyozJs-4'
sheet_name = 'PX Hominins'
folder_name = 'Hominins'
```

## Output Structure
The script creates a timestamped CSV file in the specified folder:
```
/content/drive/Shareddrives/ZooMS_Data/PRIDE/Hominins/
└── pride_metadata_aggregated_YYYYMMDD_HHMMSS.csv
```

## Output Format
The CSV includes:
- PRIDE project IDs as the first column
- All metadata fields as additional columns
- Flattened nested structures with intuitive column names
- Lists converted to comma-separated strings
- Dates in standardized datetime format

## Usage
1. Mount Google Drive and authenticate
2. Configure folder parameters if needed
3. Run the script
4. Check the output directory for the generated CSV
5. Review the console output for data summary and column information

## Data Processing Details
The script:
1. Recursively finds all non-mapped metadata JSON files
2. Processes each file:
   - Extracts PRIDE ID from filename
   - Loads and parses JSON content
   - Flattens nested structures
   - Standardizes data types
3. Combines all data into a single DataFrame
4. Exports to CSV with timestamp

## Error Handling
- Logs errors for invalid JSON files
- Skips problematic files while continuing processing
- Reports processing statistics
- Provides detailed error messages for troubleshooting

## Notes
- Files with '_mapped' suffix are excluded
- Each export creates a new timestamped file
- Column names are preserved from original metadata
- Date fields are automatically converted to datetime format

## Dependencies
- pandas: Data processing and CSV export
- pathlib: File path handling
- google.colab: Drive mounting and authentication
- tqdm: Progress tracking

#Install Dependencies

In [ ]:
# Cell 1:Import Dependencies
import os
import json
import pandas as pd
from pathlib import Path
from typing import Dict, List, Any, Optional
import logging
from datetime import datetime
from dataclasses import dataclass
from collections import defaultdict
from google.colab import drive, auth
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from tqdm.auto import tqdm

In [ ]:
# Cell 2 (Code) - Drive Mounting and Initial Setup
# def setup_drive():
#     """Mount Google Drive with proper error handling."""
#     mount_point = '/content/drive'

#     if os.path.exists(mount_point) and os.path.isdir(mount_point):
#         try:
#             os.listdir(mount_point)
#             print("Google Drive is already mounted.")
#         except:
#             print("Remounting Google Drive...")
#             drive.mount(mount_point, force_remount=True)
#     else:
#         print("Mounting Google Drive...")
#         drive.mount(mount_point)

#     # Authenticate for Google services
#     print("Authenticating...")
#     auth.authenticate_user()

# Mount Google Drive
mount_point = '/content/drive'

if os.path.exists(mount_point) and os.path.isdir(mount_point):
    try:
        os.listdir(mount_point)
        print("Google Drive is already mounted.")
    except:
        print("Mounting Google Drive...")
        drive.mount(mount_point)
else:
    print("Mounting Google Drive...")
    drive.mount(mount_point)

Google Drive is already mounted.


#Make Selections

In [ ]:
# Cell 3 (Code) - Configuration
shared_drive_base_dir_str = "/content/drive/Shareddrives/ZooMS_Data/PRIDE"  # @param {type:"string"}
folder_name = "Hominins"  # @param {type:"string"}

# Create base directory if it doesn't exist
base_dir = Path(shared_drive_base_dir_str) / folder_name
base_dir.mkdir(parents=True, exist_ok=True)

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

In [ ]:
# # Cell 3 (Code) - Configuration
# # --- Module Parameters ---
# shared_drive_base_dir_str = "/content/drive/Shareddrives/ZooMS_Data/PRIDE"  # @param {type:"string"}
# spreadsheet_id = '127K6zdl5y46DRqUwRr-V32nUDoceaddbhG9XyozJs-4'  # @param {type:"string"}
# sheet_name = 'PX Hominins'  # @param {type:"string"}
# folder_name = 'Hominins'  # @param {type:"string"}

# # Create base directory if it doesn't exist
# os.makedirs(shared_drive_base_dir_str, exist_ok=True)
# os.chdir(shared_drive_base_dir_str)

# # Configure logging
# logging.basicConfig(
#     level=logging.INFO,
#     format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
# )
# logger = logging.getLogger(__name__)

In [ ]:
# Cell 4 (Code) - Class Definitions
@dataclass
class MetadataConfig:
    """Configuration for metadata processing."""
    base_dir: str
    folder_name: str
    timestamp_format: str = "%Y%m%d_%H%M%S"
    date_columns: List[str] = None
    ignored_fields: List[str] = None

    def __post_init__(self):
        self.date_columns = self.date_columns or ['publicationDate', 'submissionDate', 'updatedDate']
        self.ignored_fields = self.ignored_fields or ['_mapped']
        self.full_path = Path(self.base_dir) / self.folder_name

# class MetadataProcessor:
#     """Handles PRIDE metadata processing and aggregation."""

#     def __init__(self, config: MetadataConfig):
#         self.config = config
#         self.logger = logging.getLogger(__name__)

#         # Create logs directory
#         self.logs_dir = self.config.full_path / 'logs'
#         self.logs_dir.mkdir(parents=True, exist_ok=True)

#         self._setup_logging()

#     def _setup_logging(self):
#         """Configure logging with file and console handlers."""
#         log_file = self.logs_dir / f"metadata_processing_{datetime.now().strftime(self.config.timestamp_format)}.log"
#         file_handler = logging.FileHandler(log_file)
#         console_handler = logging.StreamHandler()

#         formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
#         file_handler.setFormatter(formatter)
#         console_handler.setFormatter(formatter)

#         self.logger.addHandler(file_handler)
#         self.logger.addHandler(console_handler)
#         self.logger.setLevel(logging.INFO)

#     def find_metadata_files(self) -> List[Path]:
#         """Find metadata JSON files in PXD folders."""
#         metadata_files = []

#         # Check main directory
#         if not self.config.full_path.exists():
#             self.logger.error(f"Directory does not exist: {self.config.full_path}")
#             return []

#         # Look for metadata files in PXD folders with specific naming pattern
#         for pxd_dir in self.config.full_path.glob("PXD*"):
#             if pxd_dir.is_dir():
#                 expected_file = pxd_dir / f"{pxd_dir.name}_metadata.json"
#                 if expected_file.exists():
#                     self.logger.info(f"Found metadata file: {expected_file}")
#                     metadata_files.append(expected_file)
#                 else:
#                     self.logger.warning(f"No metadata file found in {pxd_dir.name}")

#         self.logger.info(f"Found {len(metadata_files)} metadata files in PXD folders")
#         return metadata_files

#     #staticmethod
#     def flatten_dict(d: Dict, parent_key: str = '', sep: str = '_') -> Dict:
#         """Flatten nested dictionary with improved handling of complex structures."""
#         items = []

#         # Special handling for file statistics
#         if parent_key == 'fileStats' and isinstance(d, dict):
#             for category, stats in d.get('filesByCategory', {}).items():
#                 items.extend([
#                     (f"files_{category}_count", stats['count']),
#                     (f"files_{category}_size_bytes", stats['totalSize']),
#                     (f"files_{category}_downloads", stats['totalDownloads'])
#                 ])
#             if 'totalFiles' in d:
#                 items.append(('total_files', d['totalFiles']))
#             if 'totalDownloads' in d:
#                 items.append(('total_downloads', d['totalDownloads']))
#             return dict(items)

#         for k, v in d.items():
#             new_key = f"{parent_key}{sep}{k}" if parent_key else k

#             if isinstance(v, dict):
#                 items.extend(MetadataProcessor.flatten_dict(v, new_key, sep=sep).items())
#             elif isinstance(v, list):
#                 if v and isinstance(v[0], dict):
#                     # Extract meaningful values from list of dictionaries
#                     values = []
#                     for item in v:
#                         if isinstance(item, dict):
#                             for field in ['name', 'value', 'accession', 'id']:
#                                 if field in item:
#                                     values.append(str(item[field]))
#                                     break
#                             else:
#                                 values.append(str(next(iter(item.values()))))
#                         else:
#                             values.append(str(item))
#                     items.append((new_key, ' | '.join(values)))
#                 else:
#                     items.append((new_key, ' | '.join(map(str, v))))
#             else:
#                 items.append((new_key, v))

#         return dict(items)

# def load_metadata_file(self, file_path: Path) -> Optional[Dict]:
#     """Load and validate metadata JSON file."""
#     try:
#         with open(file_path, 'r', encoding='utf-8') as f:
#             data = json.load(f)

#             # If data is a list of file metadata, aggregate it
#             if isinstance(data, list):
#                 # Create aggregated metadata
#                 aggregated = {
#                     "projectAccessions": data[0]["projectAccessions"] if data else [],
#                     "files": data,  # Keep all file information
#                     "submissionDate": data[0]["submissionDate"] if data else None,
#                     "publicationDate": data[0]["publicationDate"] if data else None,
#                     "updatedDate": data[0]["updatedDate"] if data else None,
#                     # Add file statistics
#                     "fileStats": {
#                         "totalFiles": len(data),
#                         "totalDownloads": sum(file.get("totalDownloads", 0) for file in data),
#                         "filesByCategory": {}
#                     }
#                 }

#                 # Calculate file statistics by category
#                 for file in data:
#                     category = file.get("fileCategory", {}).get("value", "UNKNOWN")
#                     if category not in aggregated["fileStats"]["filesByCategory"]:
#                         aggregated["fileStats"]["filesByCategory"][category] = {
#                             "count": 0,
#                             "totalSize": 0,
#                             "totalDownloads": 0
#                         }
#                     stats = aggregated["fileStats"]["filesByCategory"][category]
#                     stats["count"] += 1
#                     stats["totalSize"] += file.get("fileSizeBytes", 0)
#                     stats["totalDownloads"] += file.get("totalDownloads", 0)

#                 return aggregated
#             elif isinstance(data, dict):
#                 return data
#             else:
#                 self.logger.warning(f"Invalid metadata format in {file_path}")
#                 return None
#     except json.JSONDecodeError as e:
#         self.logger.error(f"JSON parsing error in {file_path}: {e}")
#         return None
#     except Exception as e:
#         self.logger.error(f"Error reading {file_path}: {e}")
#         return None

#     def process_metadata(self) -> pd.DataFrame:
#         """Process all metadata files into a DataFrame."""
#         metadata_files = self.find_metadata_files()
#         if not metadata_files:
#             self.logger.error(f"No metadata files found in {self.config.full_path}")
#             return pd.DataFrame()

#         all_data = []
#         for file_path in tqdm(metadata_files, desc="Processing metadata files"):
#             try:
#                 pride_id = file_path.stem.replace('_metadata', '')
#                 metadata = self.load_metadata_file(file_path)

#                 if metadata:
#                     flat_metadata = self.flatten_dict(metadata)
#                     flat_metadata['pride_id'] = pride_id
#                     all_data.append(flat_metadata)

#             except Exception as e:
#                 self.logger.error(f"Error processing {file_path}: {e}")

#         if not all_data:
#             self.logger.error("No valid metadata found")
#             return pd.DataFrame()

#         df = pd.DataFrame(all_data)
#         return self._post_process_dataframe(df)

class MetadataProcessor:
    """Handles PRIDE metadata processing and aggregation."""

    def __init__(self, config: MetadataConfig):
        self.config = config
        self.logger = logging.getLogger(__name__)

        # Create logs directory
        self.logs_dir = self.config.full_path / 'logs'
        self.logs_dir.mkdir(parents=True, exist_ok=True)

        self._setup_logging()

    def _setup_logging(self):
        """Configure logging with file and console handlers."""
        log_file = self.logs_dir / f"metadata_processing_{datetime.now().strftime(self.config.timestamp_format)}.log"
        file_handler = logging.FileHandler(log_file)
        console_handler = logging.StreamHandler()

        formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
        file_handler.setFormatter(formatter)
        console_handler.setFormatter(formatter)

        self.logger.addHandler(file_handler)
        self.logger.addHandler(console_handler)
        self.logger.setLevel(logging.INFO)

    def find_metadata_files(self) -> List[Path]:
        """Find metadata JSON files in PXD folders."""
        metadata_files = []

        # Check main directory
        if not self.config.full_path.exists():
            self.logger.error(f"Directory does not exist: {self.config.full_path}")
            return []

        # Look for metadata files in PXD folders with specific naming pattern
        for pxd_dir in self.config.full_path.glob("PXD*"):
            if pxd_dir.is_dir():
                expected_file = pxd_dir / f"{pxd_dir.name}_metadata.json"
                if expected_file.exists():
                    self.logger.info(f"Found metadata file: {expected_file}")
                    metadata_files.append(expected_file)
                else:
                    self.logger.warning(f"No metadata file found in {pxd_dir.name}")

        self.logger.info(f"Found {len(metadata_files)} metadata files in PXD folders")
        return metadata_files

    def load_metadata_file(self, file_path: Path) -> Optional[Dict]:
        """Load and validate metadata JSON file."""
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)

                # If data is a list of file metadata, aggregate it
                if isinstance(data, list):
                    # Create aggregated metadata
                    aggregated = {
                        "projectAccessions": data[0]["projectAccessions"] if data else [],
                        "files": data,  # Keep all file information
                        "submissionDate": data[0]["submissionDate"] if data else None,
                        "publicationDate": data[0]["publicationDate"] if data else None,
                        "updatedDate": data[0]["updatedDate"] if data else None,
                        "fileStats": {
                            "totalFiles": len(data),
                            "totalDownloads": sum(file.get("totalDownloads", 0) for file in data),
                            "filesByCategory": {}
                        }
                    }

                    # Calculate file statistics by category
                    for file in data:
                        category = file.get("fileCategory", {}).get("value", "UNKNOWN")
                        if category not in aggregated["fileStats"]["filesByCategory"]:
                            aggregated["fileStats"]["filesByCategory"][category] = {
                                "count": 0,
                                "totalSize": 0,
                                "totalDownloads": 0
                            }
                        stats = aggregated["fileStats"]["filesByCategory"][category]
                        stats["count"] += 1
                        stats["totalSize"] += file.get("fileSizeBytes", 0)
                        stats["totalDownloads"] += file.get("totalDownloads", 0)

                    return aggregated
                elif isinstance(data, dict):
                    return data
                else:
                    self.logger.warning(f"Invalid metadata format in {file_path}")
                    return None
        except json.JSONDecodeError as e:
            self.logger.error(f"JSON parsing error in {file_path}: {e}")
            return None
        except Exception as e:
            self.logger.error(f"Error reading {file_path}: {e}")
            return None

    @staticmethod
    def flatten_dict(d: Dict, parent_key: str = '', sep: str = '_') -> Dict:
        """Flatten nested dictionary with improved handling of complex structures."""
        items = []

        # Special handling for file statistics
        if parent_key == 'fileStats' and isinstance(d, dict):
            for category, stats in d.get('filesByCategory', {}).items():
                items.extend([
                    (f"files_{category}_count", stats['count']),
                    (f"files_{category}_size_bytes", stats['totalSize']),
                    (f"files_{category}_downloads", stats['totalDownloads'])
                ])
            if 'totalFiles' in d:
                items.append(('total_files', d['totalFiles']))
            if 'totalDownloads' in d:
                items.append(('total_downloads', d['totalDownloads']))
            return dict(items)

        for k, v in d.items():
            new_key = f"{parent_key}{sep}{k}" if parent_key else k

            if isinstance(v, dict):
                items.extend(MetadataProcessor.flatten_dict(v, new_key, sep=sep).items())
            elif isinstance(v, list):
                if v and isinstance(v[0], dict):
                    values = []
                    for item in v:
                        if isinstance(item, dict):
                            for field in ['name', 'value', 'accession', 'id']:
                                if field in item:
                                    values.append(str(item[field]))
                                    break
                            else:
                                values.append(str(next(iter(item.values()))))
                        else:
                            values.append(str(item))
                    items.append((new_key, ' | '.join(values)))
                else:
                    items.append((new_key, ' | '.join(map(str, v))))
            else:
                items.append((new_key, v))

        return dict(items)

    def process_metadata(self) -> pd.DataFrame:
        """Process all metadata files into a DataFrame."""
        metadata_files = self.find_metadata_files()
        if not metadata_files:
            self.logger.error(f"No metadata files found in {self.config.full_path}")
            return pd.DataFrame()

        all_data = []
        for file_path in tqdm(metadata_files, desc="Processing metadata files"):
            try:
                pride_id = file_path.stem.replace('_metadata', '')
                metadata = self.load_metadata_file(file_path)

                if metadata:
                    flat_metadata = self.flatten_dict(metadata)
                    flat_metadata['pride_id'] = pride_id
                    all_data.append(flat_metadata)

            except Exception as e:
                self.logger.error(f"Error processing {file_path}: {e}")

        if not all_data:
            self.logger.error("No valid metadata found")
            return pd.DataFrame()

        df = pd.DataFrame(all_data)
        return self._post_process_dataframe(df)

    def _post_process_dataframe(self, df: pd.DataFrame) -> pd.DataFrame:
        """Apply post-processing to the DataFrame."""
        # Ensure pride_id is first column
        cols = ['pride_id'] + [col for col in df.columns if col != 'pride_id']
        df = df[cols]

        # Convert date columns
        for col in self.config.date_columns:
            if col in df.columns:
                try:
                    df[col] = pd.to_datetime(df[col])
                except Exception as e:
                    self.logger.warning(f"Failed to convert {col} to datetime: {e}")

        return df

    def export_dataframe(self, df: pd.DataFrame) -> Path:
        """Export DataFrame with timestamp."""
        timestamp = datetime.now().strftime(self.config.timestamp_format)
        filename = f"pride_metadata_aggregated_{timestamp}.csv"
        output_path = self.config.full_path / filename

        try:
            df.to_csv(output_path, index=False, encoding='utf-8')
            self.logger.info(f"Data exported to {output_path}")
            return output_path
        except Exception as e:
            self.logger.error(f"Export failed: {e}")
            raise

    def generate_summary(self, df: pd.DataFrame) -> Dict[str, Any]:
        """Generate comprehensive summary statistics."""
        return {
            "total_projects": len(df),
            "total_fields": len(df.columns),
            "date_range": {
                col: {
                    "min": df[col].min(),
                    "max": df[col].max()
                } for col in self.config.date_columns if col in df.columns
            },
            "field_stats": {
                col: {
                    "type": str(df[col].dtype),
                    "non_null": df[col].count(),
                    "unique_values": df[col].nunique()
                } for col in df.columns
            }
        }


In [ ]:
# Cell 5 (Code) - Main Function
def main():
    """Main execution function."""
    print("Starting PRIDE metadata aggregation...")

    try:
        # Create configuration
        config = MetadataConfig(
            base_dir=shared_drive_base_dir_str,
            folder_name=folder_name
        )

        print("Initializing metadata processor...")
        processor = MetadataProcessor(config)

        df = processor.process_metadata()
        if not df.empty:
            output_path = processor.export_dataframe(df)
            summary = processor.generate_summary(df)

            print(f"\nProcessing complete!")
            print(f"Output file: {output_path}")
            print(f"\nSummary:")
            print(f"Total projects: {summary['total_projects']}")
            print(f"Total metadata fields: {summary['total_fields']}")

            print("\nField statistics:")
            for field, stats in summary['field_stats'].items():
                print(f"\n{field}:")
                print(f"  Type: {stats['type']}")
                print(f"  Non-null count: {stats['non_null']}")
                print(f"  Unique values: {stats['unique_values']}")

    except Exception as e:
        logger.error(f"Processing failed: {e}")
        raise

#Run Script

In [ ]:
# Cell 6 (Code) - Execution
if __name__ == "__main__":
    main()

2025-02-22 14:33:43,428 - __main__ - INFO - Found metadata file: /content/drive/Shareddrives/ZooMS_Data/PRIDE/Hominins/PXD018721/PXD018721_metadata.json
2025-02-22 14:33:43,428 - __main__ - INFO - Found metadata file: /content/drive/Shareddrives/ZooMS_Data/PRIDE/Hominins/PXD018721/PXD018721_metadata.json
2025-02-22 14:33:43,428 - __main__ - INFO - Found metadata file: /content/drive/Shareddrives/ZooMS_Data/PRIDE/Hominins/PXD018721/PXD018721_metadata.json
INFO:__main__:Found metadata file: /content/drive/Shareddrives/ZooMS_Data/PRIDE/Hominins/PXD018721/PXD018721_metadata.json
2025-02-22 14:33:43,436 - __main__ - INFO - Found metadata file: /content/drive/Shareddrives/ZooMS_Data/PRIDE/Hominins/PXD020530/PXD020530_metadata.json
2025-02-22 14:33:43,436 - __main__ - INFO - Found metadata file: /content/drive/Shareddrives/ZooMS_Data/PRIDE/Hominins/PXD020530/PXD020530_metadata.json
2025-02-22 14:33:43,436 - __main__ - INFO - Found metadata file: /content/drive/Shareddrives/ZooMS_Data/PRIDE/Ho

Starting PRIDE metadata aggregation...
Initializing metadata processor...


Processing metadata files:   0%|          | 0/8 [00:00<?, ?it/s]

2025-02-22 14:33:43,586 - __main__ - INFO - Data exported to /content/drive/Shareddrives/ZooMS_Data/PRIDE/Hominins/pride_metadata_aggregated_20250222_143343.csv
2025-02-22 14:33:43,586 - __main__ - INFO - Data exported to /content/drive/Shareddrives/ZooMS_Data/PRIDE/Hominins/pride_metadata_aggregated_20250222_143343.csv
2025-02-22 14:33:43,586 - __main__ - INFO - Data exported to /content/drive/Shareddrives/ZooMS_Data/PRIDE/Hominins/pride_metadata_aggregated_20250222_143343.csv
INFO:__main__:Data exported to /content/drive/Shareddrives/ZooMS_Data/PRIDE/Hominins/pride_metadata_aggregated_20250222_143343.csv



Processing complete!
Output file: /content/drive/Shareddrives/ZooMS_Data/PRIDE/Hominins/pride_metadata_aggregated_20250222_143343.csv

Summary:
Total projects: 8
Total metadata fields: 23

Field statistics:

pride_id:
  Type: object
  Non-null count: 8
  Unique values: 8

projectAccessions:
  Type: object
  Non-null count: 8
  Unique values: 8

files:
  Type: object
  Non-null count: 8
  Unique values: 8

submissionDate:
  Type: datetime64[ns, UTC]
  Non-null count: 8
  Unique values: 8

publicationDate:
  Type: datetime64[ns, UTC]
  Non-null count: 8
  Unique values: 8

updatedDate:
  Type: datetime64[ns, UTC]
  Non-null count: 8
  Unique values: 8

files_OTHER_count:
  Type: float64
  Non-null count: 5
  Unique values: 4

files_OTHER_size_bytes:
  Type: float64
  Non-null count: 5
  Unique values: 5

files_OTHER_downloads:
  Type: float64
  Non-null count: 5
  Unique values: 4

files_RAW_count:
  Type: int64
  Non-null count: 8
  Unique values: 6

files_RAW_size_bytes:
  Type: int64